In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

In [2]:
file = pd.read_csv('file.csv', usecols = ['date', 'user', 'pc', 'activity', 'to_removable_media', 'from_removable_media'])
device = pd.read_csv('device.csv', usecols = ['date', 'user', 'pc', 'activity'])

In [3]:
#functions to parse data

#determine if actions are performed in working hours
def is_working_hours(timestamp):
    time = timestamp.time()
    
    start = datetime.strptime('09:00:00', '%H:%M:%S').time()
    end = datetime.strptime('17:00:00', '%H:%M:%S').time()
    
    # Check if it's a weekday (Monday: 0, Tuesday: 1, ..., Friday: 4)
    if timestamp.weekday() in range(0, 5):
        if (time >= start and time <= end): # Check if it's 9 AM or later
            return "Work Hours"
        return "Off Hours"
    else:
        return 'Weekend'  # It's not a weekday

In [4]:
file['date'] = pd.to_datetime(file['date'], format= '%m/%d/%Y %H:%M:%S')
device['date'] = pd.to_datetime(device['date'], format= '%m/%d/%Y %H:%M:%S')

In [5]:
file['working_hours'] = file.date.apply(is_working_hours)
device['working_hours'] = device.date.apply(is_working_hours)

In [6]:
grouped_file = file.groupby('user')

In [7]:
#create a dataframe to hold the final parsed values
pd.set_option('display.max_rows', None)
colnames = [
    'user', 'numPCwithUSBDay', 'numPCwithUSBNight', 'numConnectionDay', 
    'numConnectionNight', 'numCopy2DeviceDay', 'numCopy2DeviceNight', 'numWrite2DeviceDay',
    'numWrite2DeviceNight', 'numCopyFromDeviceDay', 'numCopyFromDeviceNight', 'numWriteFromDeviceDay',
    'numWriteFromDeviceNight', 'numDelFromDeviceDay', 'numDelFromDeviceNight', 'numOpenOnPCDay',
    'numOpenOnPCNight'
]

results = pd.DataFrame(columns = colnames)

device_users = device.user.unique().tolist()

for user, group in grouped_file:
    
    numConnectionDay = 0
    numConnectionNight = 0
    
    if user in device_users:
        
        current_device_user = device[device.user == user]
        
        numConnectionDay = len(current_device_user.loc[(current_device_user.working_hours == 'Work Hours') & (current_device_user.activity == "Connect")])
        numConnectionNight = len(current_device_user.loc[(current_device_user.working_hours != 'Work Hours') & (current_device_user.activity == "Connect")])
    
    
    attributes = {
        'user' : user,
        'numPCwithUSBDay' : len(group.loc[(group.working_hours == 'Work Hours') & ((group.to_removable_media == True) | (group.from_removable_media == True))]),
        'numPCwithUSBNight' : len(group.loc[(group.working_hours != 'Work Hours') & ((group.to_removable_media == True) | (group.from_removable_media == True))]),
        'numConnectionDay' : numConnectionDay,
        'numConnectionNight' : numConnectionNight,
        'numCopy2DeviceDay' : len(group.loc[(group.working_hours == 'Work Hours') & (group.activity == 'File Copy') & (group.to_removable_media == True)]),
        'numCopy2DeviceNight' : len(group.loc[(group.working_hours != 'Work Hours') & (group.activity == 'File Copy') & (group.to_removable_media == True)]),
        'numWrite2DeviceDay' : len(group.loc[(group.working_hours == 'Work Hours') & (group.activity == 'File Write') & (group.to_removable_media == True)]),
        'numWrite2DeviceNight' : len(group.loc[(group.working_hours != 'Work Hours') & (group.activity == 'File Write') & (group.to_removable_media == True)]),
        'numCopyFromDeviceDay' : len(group.loc[(group.working_hours == 'Work Hours') & (group.activity == 'File Copy') & (group.from_removable_media == True)]),
        'numCopyFromDeviceNight' : len(group.loc[(group.working_hours != 'Work Hours') & (group.activity == 'File Copy') & (group.from_removable_media == True)]),
        'numWriteFromDeviceDay' : len(group.loc[(group.working_hours == 'Work Hours') & (group.activity == 'File Write') & (group.from_removable_media == True)]),
        'numWriteFromDeviceNight' : len(group.loc[(group.working_hours != 'Work Hours') & (group.activity == 'File Write') & (group.from_removable_media == True)]),
        'numDelFromDeviceDay' : len(group.loc[(group.working_hours == 'Work Hours') & (group.activity == 'File Delete') & (group.from_removable_media == True)]),
        'numDelFromDeviceNight' : len(group.loc[(group.working_hours != 'Work Hours') & (group.activity == 'File Delete') & (group.from_removable_media == True)]),
        'numOpenOnPCDay' : len(group.loc[(group.working_hours == 'Work Hours') & (group.activity == 'File Open')]),
        'numOpenOnPCNight' : len(group.loc[(group.working_hours != 'Work Hours') & (group.activity == 'File Open')]),
    }
    
    results.loc[len(results)] = [attributes[column] for column in colnames]

In [8]:
results

,user,numPCwithUSBDay,numPCwithUSBNight,numConnectionDay,numConnectionNight,numCopy2DeviceDay,numCopy2DeviceNight,numWrite2DeviceDay,numWrite2DeviceNight,numCopyFromDeviceDay,numCopyFromDeviceNight,numWriteFromDeviceDay,numWriteFromDeviceNight,numDelFromDeviceDay,numDelFromDeviceNight,numOpenOnPCDay,numOpenOnPCNight
0,AAB0162,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,13
1,AAB0398,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,4
2,AAC0610,1512,148,300,51,335,34,214,21,420,36,0,0,370,34,349,38
3,AAC0668,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,8
4,AAC3270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0
5,AAD2188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3
6,AAD3030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164,47
7,AAF0819,2475,564,830,202,515,131,317,73,688,146,0,0,637,141,610,151
8,AAF3937,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0
9,AAH0721,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,24


In [9]:
results.to_csv('cleaned_device_dataset.csv')

In [18]:
len(file.loc[(file.working_hours != 'Work Hours') & (file.activity == 'File Write') & (file.from_removable_media == True)])

0